In [1]:
import os.path as osp
import numpy as np
from scipy.stats import ttest_ind
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, recall_score, precision_score
from sklearn.feature_selection import SelectFpr

In [2]:
project_folder = '/data/shared/ptsdchild'
analysis_folder = osp.join(project_folder, 'analysis', 'resting_state', 'patients_ptsdchild_maps_dim70.gigica')
ml_folder = osp.join(analysis_folder, 'ml_analysis')
between_folder = osp.join(analysis_folder, 'group_comparison', 'between_networks')

# Random seed used in the actual analysis: used here for consistency
random_state = 1588353789

n_repeats, n_splits = 50, 5

In [3]:
# p_values are stored as 1-p
alpha = 0.95

df_correlations = pd.read_csv(osp.join(between_folder, 'unique_correlation_matrix.csv'), header=None)
print('n_subj: {}; n_connections: {}'.format(*df_correlations.shape))
df_result = pd.read_csv(osp.join(between_folder, 
                                 'palm_responder_vs_nonresponder_30perc_dat_tstat_mcfwep_m1_c2.csv'),
                        header=None)
result = df_result.to_numpy().squeeze()
id_significant = result > alpha
p_max = 1 - result[id_significant][0]
print('n_significant: {}: p_FWE: {}'.format(id_significant.sum(), p_max))
significant_connection = df_correlations.to_numpy()[:, id_significant].squeeze()

df_design = pd.read_csv(osp.join(analysis_folder, 'design', 'design_responder_30perc.csv'))
assert df_design.shape[0] == significant_connection.size

# Responders coded as 1; Non-Responders coded as 0
y_true = np.zeros(significant_connection.size)
y_true[df_design['responder_30perc_1.0'] == 1] = 1
y_true[df_design['responder_30perc_0.0'] == 1] = 0
print(np.unique(y_true, return_counts=True))

X_connection = significant_connection[:, np.newaxis]

n_subj: 40; n_connections: 1128
n_significant: 1: p_FWE: 0.012299999999999978
(array([0., 1.]), array([19, 21]))


In [4]:
def specificity(y, y_pred, **kwargs):
    # recall == sensitivity
    # recall for other class (0) == specificity
    return recall_score(y_true=y, y_pred=y_pred, pos_label=0)


def negative_predictive_value(y, y_pred, **kwargs):
    # precision == positive predictive value
    # precision for other class (0) == negative predictive value
    return precision_score(y_true=y, y_pred=y_pred, pos_label=0)


def print_results(df_results, analysis_str):
    metrics_rename = {'test_ACC': 'Balanced Accuracy',
                      'test_SENS': 'Sensitivity',
                      'test_SPEC': 'Specificity',
                      'test_PPV': 'Positive Predictive Value',
                      'test_NPV': 'Negative Predictive Value'}
    df_results = df_results[metrics_rename.keys()].copy()
    df_results.rename(columns=metrics_rename, inplace=True)
    print(f'{analysis_str}')
    print()
    print('Average performance metrics:')
    print(df_results.mean())
    print()
    print('SD performance metrics:')
    print(df_results.std())
    
    
def get_cv(random_state):
    # CV structure as in main analysis
    return RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)


def ttest_func(X, y):
    return ttest_ind(X[y == 1], X[y == 0], equal_var=False)


def get_classifier(random_state, with_univariate_ftr_selection=False):
    if with_univariate_ftr_selection:
        # unbiased classifier which includes feature selection on the training set
        clf = make_pipeline(SelectFpr(ttest_func, alpha=0.05),
                            MinMaxScaler(feature_range=(-1, 1)), 
                            LinearSVC(random_state=random_state + 1, max_iter=2000, class_weight='balanced'))
    else:
        # classifier as in main analysis
        clf = make_pipeline(MinMaxScaler(feature_range=(-1, 1)), 
                            LinearSVC(random_state=random_state + 1, max_iter=2000, class_weight='balanced'))
    return clf

### 1. Using the group-level finding as feature for classification. This is wrong as this is essentially double dipping, i.e. we take the group-level results (found on the entire sample) and perform classification (i.e. dividing data into training/testing set, etc.) with it. This is leads to a positively biased performance estimate.

In [5]:
# same classification setup as in the main analysis
cv = get_cv(random_state)
clf = get_classifier(random_state + 1)
spec = make_scorer(specificity)
npv = make_scorer(negative_predictive_value)
scores_incorrect = cross_validate(clf, X_connection, y_true, scoring={'AUC': 'roc_auc',
                                                                      'ACC': 'balanced_accuracy',
                                                                      'SENS': 'recall', # recall == sensitivity
                                                                      'SPEC': spec,
                                                                      'PPV': 'precision',
                                                                      'NPV': npv},
                        cv=cv, n_jobs=5, error_score='raise', verbose=1)
df_scores_incorrect = pd.DataFrame(scores_incorrect)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  50 tasks      | elapsed:    1.1s
[Parallel(n_jobs=5)]: Done 250 out of 250 | elapsed:    1.3s finished


In [6]:
print_results(df_scores_incorrect, "Performance via 'double dipping'")

Performance via 'double dipping'

Average performance metrics:
Balanced Accuracy            0.789433
Sensitivity                  0.822200
Specificity                  0.756667
Positive Predictive Value    0.820024
Negative Predictive Value    0.819362
dtype: float64

SD performance metrics:
Balanced Accuracy            0.130157
Sensitivity                  0.193720
Specificity                  0.237494
Positive Predictive Value    0.160601
Negative Predictive Value    0.193168
dtype: float64


### 2. This positive bias in performance can also be seen for completely random data

In [7]:
# Alpha cannot be fully FWE corrected via Bonferroni as then we won't significant features here
alpha = 0.05 

# Create random data
X_rand = np.random.rand(40, 1128)
y_rand = np.concatenate((np.zeros(19), np.ones(21)))
np.random.shuffle(y_rand)

# Run t-test on entire sample and select features which are p < alpha (= 0.05)
t, p = ttest_ind(X_rand[y_rand == 1], X_rand[y_rand == 0], equal_var=False)
X_rand_selected = X_rand[:, p < alpha]

# Perform classificaiton only with the selected features
cv = get_cv(random_state)
clf = get_classifier(random_state + 1)
spec = make_scorer(specificity)
npv = make_scorer(negative_predictive_value)
scores_rand = cross_validate(clf, X_rand_selected, y_rand, scoring={'AUC': 'roc_auc',
                                                                    'ACC': 'balanced_accuracy',
                                                                    'SENS': 'recall',
                                                                    'SPEC': spec,
                                                                    'PPV': 'precision',
                                                                    'NPV': npv},
                        cv=cv, n_jobs=5, error_score='raise', verbose=1)
df_scores_rand = pd.DataFrame(scores_rand)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 250 out of 250 | elapsed:    0.2s finished


In [8]:
print_results(df_scores_rand, "Performance via 'double dipping' in random data")

Performance via 'double dipping' in random data

Average performance metrics:
Balanced Accuracy            0.999333
Sensitivity                  1.000000
Specificity                  0.998667
Positive Predictive Value    0.999333
Negative Predictive Value    1.000000
dtype: float64

SD performance metrics:
Balanced Accuracy            0.010541
Sensitivity                  0.000000
Specificity                  0.021082
Positive Predictive Value    0.010541
Negative Predictive Value    0.000000
dtype: float64


### 3. Now we will repeat the same procedure but try to make it more correct (i.e. group-level testing within the training set only). We will apply this approach to the real data. Again we cannot use an alpha which FWE corrected since we have only a smaller data set (training set) to work with and cannot use synchronized permutation for FWE correction (but only Bonferroni) as it would be too computationally expensive.

In [9]:
# Take ALL the correlational features instead of just the group-level selected one
X_corr = df_correlations.to_numpy()

# Repeat the same classification pipeline with the addition of a t-test for each training set to select features 
# p < alpha (= 0.05)
cv = get_cv(random_state)
clf = get_classifier(random_state + 1, with_univariate_ftr_selection=True)

spec = make_scorer(specificity)
npv = make_scorer(negative_predictive_value)
scores_correct = cross_validate(clf, X_corr, y_true, scoring={'AUC': 'roc_auc',
                                                              'ACC': 'balanced_accuracy',
                                                              'SENS': 'recall',
                                                              'SPEC': spec,
                                                              'PPV': 'precision',
                                                              'NPV': npv},
                        cv=cv, n_jobs=5, error_score='raise', verbose=1)
df_scores_correct = pd.DataFrame(scores_correct)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 250 out of 250 | elapsed:    0.3s finished


In [10]:
print_results(df_scores_correct, "Unbiased performance estimate in real data")

Unbiased performance estimate in real data

Average performance metrics:
Balanced Accuracy            0.624867
Sensitivity                  0.661400
Specificity                  0.588333
Positive Predictive Value    0.649095
Negative Predictive Value    0.641771
dtype: float64

SD performance metrics:
Balanced Accuracy            0.140146
Sensitivity                  0.231027
Specificity                  0.217236
Positive Predictive Value    0.165881
Negative Predictive Value    0.216464
dtype: float64


#### We see that the observed performance drops from 78.9% accuracy to 62.5% (balanced) accuracy showing that indeed there was a huge positive bias in the initial ('double dipped') performance estimate